In [2]:
import torch
import torch.nn as nn

In [3]:
NUM_JOINS = 10

INPUT_SIZE = 16 # number of nodes per frame
SEQUENCE_LENGTH = 10 # number of frames
HIDDEN_SIZE = 2 * NUM_JOINS
NUM_LAYERS = 2 # number of RNNs to stack
NUM_CLASSES = 18 # number of categories

LEARNING_RATE = 0.1

In [4]:
TIME_DIM = 0
BATCH_DIM = 1
COORD_DIM = 2
XY_DIM = 3

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
print(device)

cpu


In [12]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, X):
        h0 = torch.zeros(self.num_layers, X.size(BATCH_DIM), self.hidden_size).to(device)

        out, _ = self.rnn(X, h0)
        # out: batch x time x hidden
        # out: (N, 28, 128)
        out = out[:, -1, :]
        # out (N, 128)
        out = self.fc(out)
        return out

In [13]:
model = RNN(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, NUM_CLASSES)

In [16]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [ ]:
# time x batch x coords (2xn matrix)